In [ ]:
import netgen.gui
from ngsolve import *
from ngsolve import curl, dx
from ngsolve import BilinearForm, GridFunction, TaskManager
from ngsolve import Preconditioner, solvers
from netgen.geom2d import unit_square
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

c0 = 299792458
l = 0.1
geo = SplineGeometry()
p1 = geo.AddPoint(0, 0)
p2 = geo.AddPoint(l, 0)
p3 = geo.AddPoint(l, l)
p4 = geo.AddPoint(0, l)

geo.AddSegment([p1, p2], bc="bottom")
geo.AddSegment([p2, p3], bc="right")
geo.AddSegment([p3, p4], bc="top")
geo.AddSegment([p4, p1], bc="left")


# A surrounding square for the PML layer
geo.AddRectangle((-0.1, 0), (0, l), bc="pml1")
geo.AddRectangle((l, 0), (l+0.1, l), bc="pml2")


mesh = Mesh(geo.GenerateMesh(maxh=l*0.5))
mesh.SetPML(pml.Cartesian((-0.1, 0, 0), (0, l, 0), alpha=1j), "pml1")
mesh.SetPML(pml.Cartesian((l, 0, 0), (l+0.1, l, 0), alpha=1j), "pml2")

Draw(mesh)

fes = HCurl(mesh, order=2, dirichlet="top", complex=True)
u, v = fes.TnT()

a = BilinearForm(y * curl(u) * curl(v) * dx)
m = BilinearForm(y * u * v * dx)

apre = BilinearForm(y * curl(u) * curl(v) * dx + y * u * v * dx)
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")

with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble()

    # build gradient matrix as sparse matrix (and corresponding scalar FESpace)
    gradmat, fesh1 = fes.CreateGradient()

    gradmattrans = gradmat.CreateTranspose()  # transpose sparse matrix
    math1 = gradmattrans @ m.mat @ gradmat  # multiply matrices
    math11 = math1.CreateSparseMatrix()
    help(math1)
#     math1[0, 0] += 1  # fix the 1-dim kernel
#     invh1 = math1.Inverse(inverse="sparsecholesky", freedofs=fesh1.FreeDofs())

#     # build the Poisson projector with operator Algebra:
#     proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat

#     projpre = proj @ pre.mat
#     evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=projpre, num=1 + 2, maxit=20,
#                                   printrates=True)

#     freq_fes = []
#     # evals[0] = 1  # <- replace nan with zero
#     for i, lam in enumerate(evals):
#         freq_fes.append(c0 * np.sqrt(np.abs(lam)) / (2 * np.pi) * 1e-6)

#     # plot results
#     gfu_E = []
#     gfu_H = []
#     for i in range(len(evecs)):
#         w = 2 * pi * freq_fes[i] * 1e6
#         gfu = GridFunction(fes)
#         gfu.vec.data = evecs[i]

#         gfu_E.append(gfu)
#         gfu_H.append(1j / (mu0 * w) * curl(gfu))


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [2]:
help(math1)

Help on ProductMatrix in module ngsolve.la object:

class ProductMatrix(BaseMatrix)
 |  Method resolution order:
 |      ProductMatrix
 |      BaseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  matA
 |  
 |  matB
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from BaseMatrix:
 |  
 |  AsVector(...)
 |      AsVector(self: ngsolve.la.BaseMatrix) -> ngsolve.la.BaseVector
 |      
 |      Interprets the matrix values as a vector
 |  
 |  CreateColVector(...)
 |      CreateColVector(self: ngsolve.la.BaseMatrix) -> ngsolve.la.BaseVector
 |  
 |  CreateDeviceMatrix(...)
 |      CreateDeviceMatrix(self: ngsolve.la.BaseMatrix) -> BaseMatrix
 |  
 